##### This script creates a file with the ticketing data on 8 mins. Place it where the Data folder is, and run it.

In [ ]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#setting start and end dates based on the other file

time_start=pd.to_datetime('23-10-2017 9:0', format="%d-%m-%Y %H:%M")
time_end=pd.to_datetime('28-02-2018 14:04', format="%d-%m-%Y %H:%M")

In [ ]:
#read, filter, and plot visitors aggregated on daily level

df=pd.read_csv('Data/1 jan 2017 tm 1 mrt 2018.csv', sep=";", encoding='latin-1')
df=df[df['Abonnement gekoppeld']=='Toegang']
df=df[['5-3-2017 13:14']]; df.columns=['Time']
df['Time']=pd.to_datetime(df['Time'], format="%d-%m-%Y %H:%M")

df=df[df['Time'] > time_start]
df=df[df['Time'] <= time_end]
df.sort_values(by='Time', ascending=True, inplace=True)
df['Date']=df['Time'].apply(lambda x: dt.datetime(x.year, x.month, x.day))

df.groupby('Date').count()['Time'].plot()

In [ ]:
#create the 8 min time intervals

time_range=time_end-time_start
num_8min_periods=round((time_range.days*24*60+time_range.seconds/60)/8)

dateList = []
for x in range (0, num_8min_periods):
    dateList.append(time_start + x*dt.timedelta(minutes = 8))

df_8min=pd.DataFrame(dateList)
df_8min.columns=['Time']

In [ ]:
#calculate cummulative number of logs from start up to the time step, assign it to 'cml' column

q=[] #temp variable that will hold cml number of visitors
for i in range (0, df_8min.shape[0]):
    q.append(df[df['Time']<=df_8min.iloc[i]['Time']].shape[0])

df_8min['cml']=q

In [ ]:
#get values for individual time steps from the cml, assign it to 'Visitors' column; plot 'Visitors'

q1=[]; q1.append(0) #temp variable that will hold number of visitors per 8 min periods
for i in range (1, df_8min.shape[0]):
    q1.append(df_8min.iloc[i]['cml']-df_8min.iloc[i-1]['cml'])

df_8min['Visitors']=q1
df_8min.drop('cml', axis=1, inplace=True)

df_8min.plot(x='Time', y='Visitors')

In [ ]:
#create the new file

df_8min.to_csv('Ticketing data on 8mins.csv', index=False)